In [1]:
%load_ext autoreload
%autoreload 2

import json
import pandas as pd
import grammar as lt
from collections import Counter
from tqdm import tqdm
from joblib import Parallel, delayed
from spacy import load
import re
import yaml
import random
from pymongo import MongoClient, UpdateOne
from plotqa import match_and_generate
nlp = load('en_core_web_sm')

2023-03-23 12:11:20.819464: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-23 12:11:21.413051: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-23 12:11:21.413097: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-23 12:11:21.413102: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
client = MongoClient()
db = client.plotqa
G = [d for d in db.captions.find({"matches.0": {"$exists": True}})]
len(G)

4

In [3]:
G

[{'_id': 14099306,
  'question_string': 'What is the difference between the Amount exported (in US$) of Goods in French Polynesia and the Amount exported (in US$) of Services in Gambia, The?',
  'template': 'arithmetic',
  'answer': 870640704.74,
  'type': 'vbar',
  'template_id': 39,
  'caption': 'The amount exported (in USD) of goods in French Polynesia exceeds the Amount exported (in USD) of Services in the Gambia by 870640704.74.',
  'matches': [{'message': 'Use “good at” to mean “skilled” with school subjects. Try this: “goods at French”.',
    'shortMessage': 'Collocation error',
    'replacements': [{'value': 'goods at French'}],
    'offset': 32,
    'length': 15,
    'context': {'text': 'The amount exported (in USD) of goods in French Polynesia exceeds the Amount exported (...',
     'offset': 32,
     'length': 15},
    'sentence': 'The amount exported (in USD) of goods in French Polynesia exceeds the Amount exported (in USD) of Services in the Gambia by 870640704.74.',
    '

In [ ]:
ignores = ['GOOD_IN_AT', 'GOO']

In [4]:
with open('/tmp/last.json', 'w') as fin:
    json.dump(G, fin, indent=2)

In [5]:
with open('/tmp/last.json', 'r') as fin:
    final = json.load(fin)

In [7]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(r['_id'], r['caption'], r.get('ignore')) for r in final)
len([g for g in res if len(g['matches']) > 0])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
2023-03-23 12:08:56.277719: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-23 12:08:56.278144: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-23 12:08:56.278159: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-23 12:08:56.286970: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror

4

In [10]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r.get('ignore', []), 'caption': r['caption']}}) for r in final]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 83,
 'nModified': 83,
 'nRemoved': 0,
 'upserted': []}

In [4]:
concise = lt.filter_lt_messages(G, message='A more concise phrase may lose no meaning and sound more powerful.')

In [7]:
[c['caption'] for c in concise]

['The Number of tourists of Madagascar in Number of arrivals is 75000.0.',
 'The Number of tourists in Madagascar in Number of departures is 39000.0.',
 'The Number of tourists of Madagascar in Number of arrivals is greater than that in Number of departures by 36000.0.',
 'The Number of tourists of Madagascar in Number of departures is less than the Number of tourists of France in Number of arrivals by 59994000.0.',
 'The difference between the Number of tourists in Madagascar in Number of departures and the Number of tourists in Lithuania in Number of arrivals is 611000.0.',
 'The ratio of the Number of tourists in Madagascar in Number of departures to that in Number of arrivals is 0.5199937600748791.',
 'The Number of tourists in European Union in Number of arrivals is 282811570.052012.',
 '284979365.008074 is the Number of tourists of European Union in Number of departures.',
 'The Number of tourists in European Union in Number of departures exceeds that in Number of arrivals by 216

In [9]:
RULE = 'ADJECTIVE_IN_ATTRIBUTE'
for c in concise:
    ig = c.get('ignore', [])
    if RULE not in ig:
        ig.append(RULE)
    c['ignore'] = ig

In [10]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(r['_id'], r['caption'], r.get('ignore')) for r in concise)
len([g for g in res if len(g['matches']) > 0])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
2023-03-23 11:20:34.279293: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-23 11:20:34.279574: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-23 11:20:34.279586: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-23 11:20:34.313444: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror

0

In [11]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r.get('ignore', [])}}) for r in concise]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 37,
 'nModified': 37,
 'nRemoved': 0,
 'upserted': []}

In [12]:
rating = lt.filter_lt_messages(G, message='The verb form ‘differ’ does not seem to match the subject ‘rating’.')
[c['caption'] for c in rating]

['The top two overall level rating differ by 4.4444444444.',
 'The top two periodicity assessment rating differ by 3.33333.',
 'The top two overall level rating differ by 4.4444444444.',
 'The top two overall level rating differ by 3.3333333333.',
 'The top two overall level rating differ by 11.1111111111.',
 'The top two periodicity assessment rating differ by 12.4122769737.',
 'The top two periodicity assessment rating differ by 6.66666.',
 'The top two overall level rating differ by 18.8888888889.',
 'The top two overall level rating differ by 3.333.',
 'The top two periodicity assessment rating differ by 3.333.',
 'The top two periodicity assessment rating differ by 3.3333333333.',
 'The top two periodicity assessment rating differ by 0.877.',
 'The top two overall level rating differ by 2.222.',
 'The top two periodicity assessment rating differ by 6.66667.',
 'The top two overall level rating differ by 3.333.',
 'The top two overall level rating differ by 0.2116402381.',
 'The to

In [13]:
for c in rating:
    c['caption'] = c['caption'].replace('rating differ', 'ratings differ')

In [15]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(r['_id'], r['caption'], r.get('ignore')) for r in rating)
len([g for g in res if len(g['matches']) > 0])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  13 out of  29 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  29 out of  29 | elapsed:    0.0s finished


0

In [16]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'caption': r['caption']}}) for r in rating]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 29,
 'nModified': 29,
 'nRemoved': 0,
 'upserted': []}

In [17]:
sent_start = lt.filter_lt_messages(G, message='If a new sentence starts here, add a space and start with an uppercase letter.')
[c['caption'] for c in sent_start]

['The total agricultural land in Switzerland is not greater than 240000 sq.km in any years.',
 'The total agricultural land in the United Kingdom is not greater than 200000 sq.km in any years.',
 'The total agricultural land in the United Arab Emirates is not greater than 150000 sq.km in any years.',
 'The total agricultural land in Gabon is greater than 34000 sq.km in 6 years.',
 'The total agricultural land in the least developed countries is greater than 6000000 sq.km in 5 years.',
 'The total agricultural land in Costa Rica is greater than 18000 sq.km in 3 years.',
 'The total agricultural land in Lithuania is not greater than 680000 sq.km in any years.',
 'The total agricultural land in Denmark is not greater than 136000 sq.km in any years.',
 'The total agricultural land in Slovenia is never more than 400000 sq.km in any years.',
 'The total agricultural land in Finland is never more than 100000 sq.km in any years.',
 'The total agricultural land in Tuvalu is not greater than 300

In [18]:
assert all(['sq.km' in c['caption'] for c in sent_start])

In [19]:
for c in sent_start:
    c['caption'] = c['caption'].replace('sq.km', 'sq km')

In [20]:
res = [lt.process(r['_id'], r['caption'], r.get('ignore')) for r in sent_start]
len([g for g in res if len(g['matches']) > 0])

0

In [21]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'caption': r['caption']}}) for r in sent_start]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 28,
 'nModified': 28,
 'nRemoved': 0,
 'upserted': []}

In [22]:
adj = lt.filter_lt_messages(G, message='Make sure that the adjective')
[c['caption'] for c in adj]

['0.5945835176 is the total elevated land area in 2000.',
 '0.3513818 is the total elevated land area in 2000.',
 '0.3752189172 is the total elevated land area in 2000.',
 '0.2082346 is the total elevated land area in 2000.',
 '5.497674 is the total elevated land area in 2000.',
 '71.98609 is the total elevated land area in 1990.',
 '8.7086348821 is the total adjusted net savings in Sub-Saharan Africa (all income levels) in the graph.',
 '189190755439.18393 is the total adjusted net savings in 1970 in the graph.',
 '50.6151606875 is the total adjusted net savings in 1972 in the graph.',
 '67.4824630833 is the total adjusted net savings in 1970 in the graph.',
 '56.4365840963 is the total adjusted net savings in 1970 in the graph.',
 '243159014617.9618 is the total adjusted net savings in 1972 in the graph.',
 '85.50290936 is the total adjusted net savings in 1970 in the graph.',
 '96.7031561347 is the total adjusted net savings in 1973 in the graph.',
 '96.5933481298 is the total adjus

In [24]:
RULE = 'ADVERB_OR_HYPHENATED_ADJECTIVE'
for c in adj:
    ig = c.get('ignore', [])
    if RULE not in ig:
        ig.append(RULE)
    c['ignore'] = ig
    
res = [lt.process(r['_id'], r['caption'], r.get('ignore')) for r in adj]
len([g for g in res if len(g['matches']) > 0])

0

In [25]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r['ignore']}}) for r in adj]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 50,
 'nModified': 50,
 'nRemoved': 0,
 'upserted': []}

In [27]:
ctr.most_common()

[('A more concise phrase may lose no meaning and sound more powerful.', 37),
 ('The verb form ‘differ’ does not seem to match the subject ‘rating’.', 29),
 ('If a new sentence starts here, add a space and start with an uppercase letter.',
  28),
 ('Make sure that the adjective ‘total’ is correct. Possibly, it should be an adverb (typically ~ly) that modifies ‘adjusted’. Possibly, it should be the first word in a compound adjective (hyphenated adjective). Possibly, it is correct.',
  26),
 ('Make sure that the adjective ‘total’ is correct. Possibly, it should be an adverb (typically ~ly) that modifies ‘elevated’. Possibly, it should be the first word in a compound adjective (hyphenated adjective). Possibly, it is correct.',
  24),
 ('Did you mean “whom”?', 22),
 ('Please check whether a word is missing between ‘The’ and ‘is’ or whether ‘The’ is the correct determiner.',
  11),
 ('Please verify that the plural noun “days” is in agreement with the quantifier “1”. Did you mean to use the s

In [4]:
least = lt.filter_lt_messages(G, message='Non-standard use of the comparative or superlative. Did you mean “least low”?')
least[0]

{'_id': 324567,
 'question_string': 'Across all years, what is the minimum lower secondary completion rate of female students in Albania ?',
 'template': 'min_max',
 'answer': 82.8153991699,
 'type': 'dot',
 'template_id': 27,
 'caption': 'The least lower secondary completion rate of female students in Albania across years is 82.8153991699.',
 'matches': [{'message': 'Non-standard use of the comparative or superlative. Did you mean “least low”?',
   'shortMessage': 'Grammatical problem: non-standard comparative/superlative',
   'replacements': [{'value': 'least low'}],
   'offset': 4,
   'length': 11,
   'context': {'text': 'The least lower secondary completion rate of female stu...',
    'offset': 4,
    'length': 11},
   'sentence': 'The least lower secondary completion rate of female students in Albania across years is 82.8153991699.',
   'type': {'typeName': 'Other'},
   'rule': {'id': 'LEAST_COMPARATIVE',
    'subId': '1',
    'sourceFile': 'grammar.xml',
    'description': "'leas

In [15]:
random.choices([c['caption'] for c in least], k=10)

['The least lower secondary completion rate of male students in Cabo Verde across years is 46.5438499451.',
 'The least lower secondary completion rate of female students in the Middle East & North Africa (all income levels) across years is 70.4866333008.',
 'The least lower secondary completion rate of male students in Algeria across years is 46.0660705566.',
 'The least lower secondary completion rate of female students in Kuwait across years is 99.4758911133.',
 'The least lower secondary completion rate of male students in Chile across years is 92.3327713013.',
 'The least lower secondary school starting age of children in 1973 across all countries is 10.0.',
 'The least lower secondary completion rate of male students in the Central African Republic across all years is 9.79974.',
 'The least lower secondary school starting age of children in 1972 across all countries is 12.0.',
 'The least lower secondary completion rate of male students in Guatemala across years is 44.2846794128.

In [16]:
len(least)

58

In [18]:
for c in least:
    c['caption'] = c['caption'].replace('The least lower secondary', 'The minimum lower secondary')

In [14]:
len([c for c in agree if c['caption'].startswith('The top two')]) / len(agree)

0.9508670520231214

In [21]:
duplicate = lt.filter_lt_messages(G, message='This phrase is duplicated.')
len(duplicate)

72

In [23]:
for c in duplicate:
    assert ('in Amount in Amount' in c['caption']) or ('in Investment in Investmen' in c['caption'])

In [24]:
for c in duplicate:
    c['caption'] = c['caption'].replace('in Amount in Amount', 'in Amount')
    c['caption'] = c['caption'].replace('in Investment in Investment', 'in Investment')

In [25]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(r['_id'], r['caption'], r.get('ignore')) for r in duplicate)
len([g for g in res if len(g['matches']) > 0])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
2023-03-23 11:07:20.037225: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-23 11:07:20.037630: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-23 11:07:20.037648: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-23 11:07:20.061890: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror

0

In [26]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'caption': r['caption']}}) for r in duplicate]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 72,
 'nModified': 72,
 'nRemoved': 0,
 'upserted': []}

In [27]:
ctr.most_common()

[('Non-standard use of the comparative or superlative. Did you mean “least low”?',
  58),
 ('This phrase is duplicated. You should probably use “in Amount” only once.',
  42),
 ('Possible agreement error. The noun ‘computer’ seems to be countable.', 41),
 ('A more concise phrase may lose no meaning and sound more powerful.', 37),
 ('This phrase is duplicated. You should probably use “in Investment” only once.',
  30),
 ('The verb form ‘differ’ does not seem to match the subject ‘rating’.', 29),
 ('If a new sentence starts here, add a space and start with an uppercase letter.',
  28),
 ('Make sure that the adjective ‘total’ is correct. Possibly, it should be an adverb (typically ~ly) that modifies ‘adjusted’. Possibly, it should be the first word in a compound adjective (hyphenated adjective). Possibly, it is correct.',
  26),
 ('Make sure that the adjective ‘total’ is correct. Possibly, it should be an adverb (typically ~ly) that modifies ‘elevated’. Possibly, it should be the first wo

In [28]:
agree = lt.filter_lt_messages(G, message='Possible agreement error')
len(agree)

44

In [32]:
for c in agree:
    c['caption'] = c['caption'].replace('The top two computer and other services differs by', 'The top two computer and other services differ by')

In [33]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(r['_id'], r['caption'], r.get('ignore')) for r in agree)
len([g for g in res if len(g['matches']) > 0])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  36 out of  44 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  44 out of  44 | elapsed:    0.1s finished


44

In [35]:
for c in agree:
    if c['caption'].startswith('The top two computer and other services differ by'):
        ig = c.get('ignore', [])
        if 'CD_NN' not in ig:
            ig.append('CD_NN')
        c['ignore'] = ig

In [36]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'caption': r['caption']}}) for r in agree]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 44,
 'nModified': 41,
 'nRemoved': 0,
 'upserted': []}

In [37]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r.get('ignore', [])}}) for r in agree]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 44,
 'nModified': 44,
 'nRemoved': 0,
 'upserted': []}

In [15]:
top2 = [c for c in agree if c['caption'].startswith('The top two')]
top2[0]['caption']

'The top two rural population density differ by 90002.'

In [16]:
doc = nlp(_)
doc

The top two rural population density differ by 90002.

ValueError: Tag for verb . not valid.

In [5]:
with open('/tmp/repbulic.txt', 'w') as fout:
    fout.write('\n'.join([c['caption'] for c in republic]))

In [6]:
doc = nlp('The highest and the second-highest net disbursements made by the Slovak Republic differ by 1530000.')
root, nsubj = lt.find_root_nsubj(doc)

In [18]:
qids = [s['_id'] for s in top2]
sents = [s['caption'] for s in top2]

pairs = []
for doc, qid in zip(nlp.pipe(sents), qids):
    root = False
    for token in doc:
        if token.dep_ == 'ROOT':
            root = token
            break
    if root:
        try:
            child = [k for k in root.children if k.dep_ == 'nsubj'][0]
            pairs.append((qid, root.text, child.text))
        except IndexError:
            continue

pairs = pd.DataFrame(pairs, columns=['qid', 'root', 'nsubj'])
pairs.drop_duplicates(subset=['root', 'nsubj'])

,qid,root,nsubj
0,2758,differ,density
1,11035,differ,rent
2,14437,differ,population
14,80991,differ,growth
23,341533,differ,rating
28,502938,differ,production
31,511266,differ,expectancy


In [19]:
repls = [
    ('density', 'densities'),
    ('rent', 'rents'),
    ('population', 'populations'),
    ('growth', 'growths'),
    ('production', 'productions'),
    ('expectancy', 'expectancies')
]
for k in top2:
    for repl in repls:
        k['caption'] = k['caption'].replace(*repl)

In [19]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(r['_id'], r['caption'], r.get('ignore')) for r in least)
len([g for g in res if len(g['matches']) > 0])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
2023-03-23 11:02:05.101011: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-23 11:02:05.101391: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-23 11:02:05.101407: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-23 11:02:05.134428: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror

0

In [20]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'caption': r['caption']}}) for r in least]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 58,
 'nModified': 58,
 'nRemoved': 0,
 'upserted': []}

In [19]:
[(c.text, c.dep_) for c in root.children]

[('number', 'nsubj'), ('by', 'prep'), ('.', 'punct')]

In [21]:
C = 0
for c in republic:
    if re.search(r'disbursements.*differ by', c['caption']):
        C += 1
        ig = c.get('ignore', [])
        if 'HE_VERB_AGR' not in ig:
            ig.append('HE_VERB_AGR')
        c['ignore'] = ig
C

74

In [22]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r.get('ignore',  [])}}) for r in republic]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 1302,
 'nModified': 1302,
 'nRemoved': 0,
 'upserted': []}

In [23]:
pat = r'The (?P<metric>.*) (of|in) \d+ in (?P<loc>.*) and the (?P=metric) (of|in) \d+ in (?P<loc2>.*) differ by.*$'

C = 0
for c in republic:
    if re.match(pat, c['caption']):
        C += 1
        ig = c.get('ignore', [])
        if 'HE_VERB_AGR' not in ig:
            ig.append('HE_VERB_AGR')
        c['ignore'] = ig
C

991

In [24]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r.get('ignore',  [])}}) for r in republic]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 1302,
 'nModified': 990,
 'nRemoved': 0,
 'upserted': []}

In [32]:
lines = [c['caption'] for c in republic if len(c.get('ignore', [])) == 0]

In [26]:
len(lines)

238

In [33]:
with open('/tmp/repbulic.txt', 'w') as fout:
    fout.write('\n'.join(lines))

In [29]:
pat = r'^.*[T|t]he (?P<metric>.*) (of|in) (?P<grp1>.*) (of|in) (?P<loc1>.*) and the (?P=metric) (of|in) (?P<grp2>.*) (of|in) (?P<loc2>.*) differ by.*$'
C = 0
for c in republic:
    if len(c.get('ignore', [])) == 0:
        if re.match(pat, c['caption']):
            C += 1
C

234

In [30]:
C = 0
for c in republic:
    if re.match(pat, c['caption']):
        C += 1
        ig = c.get('ignore', [])
        if 'HE_VERB_AGR' not in ig:
            ig.append('HE_VERB_AGR')
        c['ignore'] = ig
C

1225

In [31]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r.get('ignore',  [])}}) for r in republic]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 1302,
 'nModified': 234,
 'nRemoved': 0,
 'upserted': []}

In [34]:
C = 0
for k in republic:
    if 'are the same' in k['caption']:
        C += 1
C

4

In [35]:
C = 0
for c in republic:
    if 'are the same' in c['caption']:
        C += 1
        ig = c.get('ignore', [])
        if 'HE_VERB_AGR' not in ig:
            ig.append('HE_VERB_AGR')
        c['ignore'] = ig
C

4

In [36]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r.get('ignore',  [])}}) for r in republic]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 1302,
 'nModified': 4,
 'nRemoved': 0,
 'upserted': []}

In [37]:
ctr.most_common()

[('The singular proper name ‘Republic’ must be used with a third-person or a past tense verb.',
  1302),
 ('The verb form ‘differ’ does not seem to match the subject ‘population’.',
  101),
 ('The verb form ‘differ’ does not seem to match the subject ‘rent’.', 64),
 ('Non-standard use of the comparative or superlative. Did you mean “least low”?',
  58),
 ('The verb form ‘differ’ does not seem to match the subject ‘birth’.', 54),
 ('The verb form ‘differ’ does not seem to match the subject ‘production’.',
  48),
 ('This phrase is duplicated. You should probably use “in Amount” only once.',
  42),
 ('Possible agreement error. The noun ‘computer’ seems to be countable.', 41),
 ('A more concise phrase may lose no meaning and sound more powerful.', 37),
 ('This phrase is duplicated. You should probably use “in Investment” only once.',
  30),
 ('The verb form ‘differ’ does not seem to match the subject ‘rating’.', 29),
 ('The verb form ‘differ’ does not seem to match the subject ‘density’.',

In [5]:
qids = [s['_id'] for s in infl]
sents = [s['caption'] for s in infl]

pairs = []
for doc, qid in zip(nlp.pipe(sents), qids):
    root = False
    for token in doc:
        if token.dep_ == 'ROOT':
            root = token
            break
    if root:
        try:
            child = [k for k in root.children if k.dep_ == 'nsubj'][0]
            pairs.append((qid, root.text, child.text))
        except IndexError:
            continue

pairs = pd.DataFrame(pairs, columns=['qid', 'root', 'nsubj'])
pairs.drop_duplicates(subset=['root', 'nsubj'])

,qid,root,nsubj
0,9882,grow,cost
1,12114,grow,amount
5,56438,increase,cost
7,62581,increase,amount
8,66132,grows,amount
11,120614,increase,percentage
14,121826,grow,debt
15,122405,grow,percentage
19,127965,grow,balance
20,128070,increase,balance


In [6]:
pairs['rootnsubj'] = pairs.apply(lambda x: f'{x["root"]}-{x["nsubj"]}', axis=1)
pairs['rootnsubj'].value_counts()

exceeds-Amount         184
exceeds-Tax             95
increase-percentage     45
grow-percentage         44
exceeds-amount          39
increase-amount         36
exceeds-percentage      31
grow-amount             30
increase-balance        30
grow-balance            23
exceeds-Adequacy        19
exceeds-emission        15
exceeds-number          11
is-mean                 11
grows-amount             9
grow-number              8
grow-rate                6
increase-number          5
grows-number             5
increase-rate            5
grow-cost                3
increase-debt            3
grow-debt                3
grows-percentage         3
grows-balance            2
grows-rate               1
increase-cost            1
exceeds-%                1
Name: rootnsubj, dtype: int64

In [8]:
for prefix in pairs['rootnsubj'].unique():
    is_number = set(pairs[pairs['rootnsubj'] == prefix]['qid'].unique())
    docs = [c['caption'] for c in G if c['_id'] in is_number]
    for doc in nlp.pipe(docs):
        try:
            agrees = lt.has_agreement(*lt.find_root_nsubj(doc))
        except (IndexError, ValueError):
            print(prefix)
            break
        if not agrees:
            print(prefix)
            break
    

is-mean
exceeds-Tax


In [31]:
to_ignore = pairs[pairs['rootnsubj'] != 'is-mean']['qid'].unique()

In [9]:
for t in infl:
    ig = t.get('ignore', [])
    if 'NNS_IN_NNP_VBZ' not in ig:
        ig.append('NNS_IN_NNP_VBZ')
    t['ignore'] = ig

    
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r['ignore']}}) for r in infl]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 709,
 'nModified': 709,
 'nRemoved': 0,
 'upserted': []}

In [10]:
ctr

Counter({'The verb form ‘differ’ does not seem to match the subject ‘density’.': 28,
         'With the plural noun ‘emissions’, the verb inflection ‘does’ is not correct.': 4,
         'The verb form ‘differ’ does not seem to match the subject ‘rent’.': 64,
         'With the plural noun ‘banks’, the verb inflection ‘does’ is not correct.': 6,
         'The verb form ‘differ’ does not seem to match the subject ‘population’.': 101,
         'With the plural noun ‘banks’, the verb inflection ‘grows’ is not correct.': 1,
         'Please verify that the plural noun “days” is in agreement with the quantifier “1”. Did you mean to use the singular form?': 6,
         'The verb form ‘differ’ does not seem to match the subject ‘growth’.': 5,
         'With the plural noun ‘services’, the verb inflection ‘does’ is not correct.': 71,
         'With the plural noun ‘tourists’, the verb inflection ‘does’ is not correct.': 47,
         'With the plural noun ‘securities’, the verb inflection ‘does’

In [11]:
to_remove = []
for k in ctr:
    if ', the verb inflection ' in k:
        to_remove.append(k)
        
for key in to_remove:
    del ctr[key]

In [14]:
sum(ctr.values()) - 1302

916

In [15]:
X = lt.filter_lt_messages(G, message='does not seem to match the subject')
X[0]

{'_id': 2758,
 'question_string': 'What is the difference between the highest and the second-highest rural population density ?',
 'template': 'compound',
 'answer': 90002,
 'type': 'dot',
 'template_id': 15,
 'caption': 'The top two rural population density differ by 90002.',
 'matches': [{'message': 'The verb form ‘differ’ does not seem to match the subject ‘density’.',
   'shortMessage': '',
   'replacements': [{'value': 'differs'},
    {'value': 'differed'},
    {'value': 'will differ'}],
   'offset': 37,
   'length': 6,
   'context': {'text': 'The top two rural population density differ by 90002.',
    'offset': 37,
    'length': 6},
   'sentence': 'The top two rural population density differ by 90002.',
   'type': {'typeName': 'Other'},
   'rule': {'id': 'SUBJECT_VERB_AGREEMENT',
    'subId': '1',
    'sourceFile': 'grammar.xml',
    'description': 'Subject-verb agreement',
    'issueType': 'grammar',
    'category': {'id': 'GRAMMAR', 'name': 'Grammar'}},
   'ignoreForIncompleteS

In [16]:
qids = [s['_id'] for s in X]
sents = [s['caption'] for s in X]

pairs = []
for doc, qid in zip(nlp.pipe(sents), qids):
    root = False
    for token in doc:
        if token.dep_ == 'ROOT':
            root = token
            break
    if root:
        try:
            child = [k for k in root.children if k.dep_ == 'nsubj'][0]
            pairs.append((qid, root.text, child.text))
        except IndexError:
            continue

pairs = pd.DataFrame(pairs, columns=['qid', 'root', 'nsubj'])
pairs.drop_duplicates(subset=['root', 'nsubj'])

,qid,root,nsubj
0,2758,differ,density
1,11035,differ,rent
2,14437,differ,population
14,80991,differ,growth
23,243331,is,withdrawals
24,341533,differ,rating
29,501788,is,flows
30,502938,differ,production
35,511266,differ,expectancy
104,594678,increases,withdrawals


In [17]:
pairs['rootnsubj'] = pairs.apply(lambda x: f'{x["root"]}-{x["nsubj"]}', axis=1)
pairs['rootnsubj'].value_counts()

is-withdrawals           223
differ-population        101
differ-rent               64
differ-expectancy         54
differ-production         48
differ-rating             29
differ-density            28
is-flows                   6
differ-growth              5
is-OECD                    2
is-income                  2
increases-withdrawals      1
Name: rootnsubj, dtype: int64

In [19]:
is_withdraw = set(pairs[pairs['rootnsubj'] == 'is-withdrawals']['qid'].values)
is_withdraw = [c for c in G if c['_id'] in is_withdraw]

In [20]:
is_withdraw[0]

{'_id': 243331,
 'question_string': 'Is the annual freshwater withdrawals in Gambia, The strictly greater than the annual freshwater withdrawals in Hungary over the years ?',
 'template': 'compound',
 'answer': 'No',
 'type': 'dot',
 'template_id': 20,
 'caption': 'The annual freshwater withdrawals in the Gambia is not strictly greater than the annual freshwater withdrawals in Hungary over the years.',
 'matches': [{'message': 'The verb form ‘is’ does not seem to match the subject ‘withdrawals’.',
   'shortMessage': '',
   'replacements': [{'value': 'are'},
    {'value': 'was'},
    {'value': 'were',
     'shortDescription': "second-person singular simple past of 'be', e.g. 'you were'"},
    {'value': 'will be'}],
   'offset': 48,
   'length': 2,
   'context': {'text': '...al freshwater withdrawals in the Gambia is not strictly greater than the annual fr...',
    'offset': 43,
    'length': 2},
   'sentence': 'The annual freshwater withdrawals in the Gambia is not strictly greater than

In [22]:
for c in is_withdraw:
    c['caption'] = c['caption'].replace('is not strictly greater than', 'are not strictly greater than')
    c['caption'] = c['caption'].replace('across all years is', 'across all years are')
    c['caption'] = c['caption'].replace('across all countries is', 'across all countries are')

In [23]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(r['_id'], r['caption'], r.get('ignore')) for r in is_withdraw)
len([g for g in res if len(g['matches']) > 0])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
2023-03-22 16:39:07.372831: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-22 16:39:07.373258: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-22 16:39:07.373273: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-22 16:39:07.385574: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror

5

In [24]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'caption': r['caption']}}) for r in is_withdraw]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 223,
 'nModified': 218,
 'nRemoved': 0,
 'upserted': []}

In [26]:
ctr.most_common()

[('The singular proper name ‘Republic’ must be used with a third-person or a past tense verb.',
  1302),
 ('The verb form ‘is’ does not seem to match the subject ‘withdrawals’.', 223),
 ('The verb form ‘differ’ does not seem to match the subject ‘population’.',
  101),
 ('The verb form ‘differ’ does not seem to match the subject ‘rent’.', 64),
 ('Non-standard use of the comparative or superlative. Did you mean “least low”?',
  58),
 ('The verb form ‘differ’ does not seem to match the subject ‘birth’.', 54),
 ('The verb form ‘differ’ does not seem to match the subject ‘production’.',
  48),
 ('This phrase is duplicated. You should probably use “in Amount” only once.',
  42),
 ('Possible agreement error. The noun ‘computer’ seems to be countable.', 41),
 ('A more concise phrase may lose no meaning and sound more powerful.', 37),
 ('This phrase is duplicated. You should probably use “in Investment” only once.',
  30),
 ('The verb form ‘differ’ does not seem to match the subject ‘rating’.'

In [12]:
ctr.most_common()

[('The singular proper name ‘Republic’ must be used with a third-person or a past tense verb.',
  1302),
 ('The verb form ‘is’ does not seem to match the subject ‘withdrawals’.', 223),
 ('The verb form ‘differ’ does not seem to match the subject ‘population’.',
  101),
 ('The verb form ‘differ’ does not seem to match the subject ‘rent’.', 64),
 ('Non-standard use of the comparative or superlative. Did you mean “least low”?',
  58),
 ('The verb form ‘differ’ does not seem to match the subject ‘birth’.', 54),
 ('The verb form ‘differ’ does not seem to match the subject ‘production’.',
  48),
 ('This phrase is duplicated. You should probably use “in Amount” only once.',
  42),
 ('Possible agreement error. The noun ‘computer’ seems to be countable.', 41),
 ('A more concise phrase may lose no meaning and sound more powerful.', 37),
 ('This phrase is duplicated. You should probably use “in Investment” only once.',
  30),
 ('The verb form ‘differ’ does not seem to match the subject ‘rating’.'

In [37]:
p_import = lt.filter_lt_messages(G, message='You should probably use “import”.')
p_import[0]

{'_id': 13408,
 'question_string': 'Across all years, what is the maximum arms imports ?',
 'template': 'min_max',
 'answer': '235000000',
 'type': 'dot',
 'template_id': 22,
 'caption': 'The greatest arms imports across all years are 235000000.',
 'matches': [{'message': 'You should probably use “import”.',
   'shortMessage': 'Possible agreement error',
   'replacements': [{'value': 'import'}],
   'offset': 18,
   'length': 7,
   'context': {'text': 'The greatest arms imports across all years are 235000000.',
    'offset': 18,
    'length': 7},
   'sentence': 'The greatest arms imports across all years are 235000000.',
   'type': {'typeName': 'Other'},
   'rule': {'id': 'AGREEMENT_SENT_START',
    'subId': '4',
    'sourceFile': 'grammar.xml',
    'description': 'Possible agreement error: plural noun + singular verb',
    'issueType': 'grammar',
    'category': {'id': 'GRAMMAR', 'name': 'Grammar'}},
   'ignoreForIncompleteSentence': True,
   'contextForSureMatch': 4}]}

In [40]:
for t in p_import:
    ig = t.get('ignore', [])
    if 'AGREEMENT_SENT_START' not in ig:
        ig.append('AGREEMENT_SENT_START')
    t['ignore'] = ig

    
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r['ignore']}}) for r in p_import]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 233,
 'nModified': 233,
 'nRemoved': 0,
 'upserted': []}

In [41]:
ctr.most_common(10)

[('The singular proper name ‘Republic’ must be used with a third-person or a past tense verb.',
  1302),
 ('You should probably use “import”.', 233),
 ('The verb form ‘is’ does not seem to match the subject ‘withdrawals’.', 223),
 ('You should probably use “export”.', 210),
 ('With the plural noun ‘materials’, the verb inflection ‘exceeds’ is not correct.',
  180),
 ('The verb form ‘differ’ does not seem to match the subject ‘population’.',
  101),
 ('With the plural noun ‘countries’, the verb inflection ‘does’ is not correct.',
  75),
 ('With the plural noun ‘services’, the verb inflection ‘does’ is not correct.',
  71),
 ('With the plural noun ‘exports’, the verb inflection ‘exceeds’ is not correct.',
  69),
 ('The verb form ‘differ’ does not seem to match the subject ‘rent’.', 64)]

In [42]:
export = lt.filter_lt_messages(G, message='You should probably use “export”.')
export[0]

{'_id': 13415,
 'question_string': 'What is the average arms exports per year?',
 'template': 'arithmetic',
 'answer': 10400000.0,
 'type': 'dot',
 'template_id': 32,
 'caption': 'The average arms exports per year are 10400000.0.',
 'matches': [{'message': 'You should probably use “export”.',
   'shortMessage': 'Possible agreement error',
   'replacements': [{'value': 'export'}],
   'offset': 17,
   'length': 7,
   'context': {'text': 'The average arms exports per year are 10400000.0.',
    'offset': 17,
    'length': 7},
   'sentence': 'The average arms exports per year are 10400000.0.',
   'type': {'typeName': 'Other'},
   'rule': {'id': 'AGREEMENT_SENT_START',
    'subId': '4',
    'sourceFile': 'grammar.xml',
    'description': 'Possible agreement error: plural noun + singular verb',
    'issueType': 'grammar',
    'category': {'id': 'GRAMMAR', 'name': 'Grammar'}},
   'ignoreForIncompleteSentence': True,
   'contextForSureMatch': 4}]}

In [43]:
[c['caption'] for c in random.choices(export, k=10)]

['The maximum arms exports across all years are 14000000.',
 'The arms exports in High income: OECD are greater than that in Australia.',
 'The greatest arms exports across all countries are 84000000.',
 'The lowest arms exports across all countries are 25000000.',
 'The average arms exports per country are 1286250000.0.',
 'The metals exports in India and that is greater than the percentage of ores by -68.1533783624.',
 'The average arms exports per country are 5211750000.0.',
 'The services exports in 1962 in Niger and that is -2839199764.0 more than the Amount of goods.',
 'The services exports in Mongolia in 2010 and that is -2717227169206.58 more than the Amount of goods.',
 'The average arms exports per country are 85500000.0.']

In [44]:
for t in export:
    ig = t.get('ignore', [])
    if 'AGREEMENT_SENT_START' not in ig:
        ig.append('AGREEMENT_SENT_START')
    t['ignore'] = ig

    
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r['ignore']}}) for r in export]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 210,
 'nModified': 210,
 'nRemoved': 0,
 'upserted': []}

In [45]:
infl = lt.filter_lt_messages(G, message='verb inflection')
Counter([i['matches'][0]['message'] for i in infl])

Counter({'With the plural noun ‘banks’, the verb inflection ‘is’ is not correct.': 168,
         'With the plural noun ‘emissions’, the verb inflection ‘is’ is not correct.': 165,
         'With the plural noun ‘emissions’, the verb inflection ‘does’ is not correct.': 4,
         'With the plural noun ‘banks’, the verb inflection ‘does’ is not correct.': 6,
         'With the plural noun ‘disbursements’, the verb inflection ‘is’ is not correct.': 87,
         'With the plural noun ‘banks’, the verb inflection ‘grows’ is not correct.': 1,
         'With the plural noun ‘services’, the verb inflection ‘does’ is not correct.': 71,
         'With the plural noun ‘tourists’, the verb inflection ‘does’ is not correct.': 47,
         'With the plural noun ‘securities’, the verb inflection ‘is’ is not correct.': 225,
         'With the plural noun ‘securities’, the verb inflection ‘does’ is not correct.': 9,
         'With the plural noun ‘visitors’, the verb inflection ‘is’ is not correct.': 

In [20]:
for doc in nlp.pipe(docs):
    assert lt.has_agreement(*lt.find_root_nsubj(doc))

In [8]:
pairs[pairs['nsubj'] == 'OECD']

,qid,root,nsubj
2239,3115452,is,OECD


In [17]:
words = lt.get_repl_values(repeat)
for k in repeat:
    msg = k['caption']
    for word in words:
        msg = re.sub(f'\\b{word} {word}\\b', f'{word}', msg, flags=re.IGNORECASE)
    k['caption'] = msg

In [18]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(r['_id'], r['caption'], r.get('ignore')) for r in repeat)
len([g for g in res if len(g['matches']) > 0])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
2023-03-21 11:47:55.787151: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-21 11:47:55.787560: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-21 11:47:55.787582: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-21 11:47:55.840225: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror

0

In [19]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'caption': r['caption']}}) for r in repeat]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 847,
 'nModified': 847,
 'nRemoved': 0,
 'upserted': []}

In [20]:
interest = lt.filter_lt_messages(G, message='The usual collocation for “Interest” is “in”, not “by”.')
interest[0]

{'_id': 142801,
 'question_string': 'In the year 1999, what is the difference between the percentage of revenue generated by grants and percentage of revenue generated by interest ?',
 'template': 'arithmetic',
 'answer': 11.5996389009,
 'type': 'dot',
 'template_id': 42,
 'caption': 'The percentage of revenue generated by grants is greater than the percentage of revenue generated by interest by 11.5996389009 in the year 1999.',
 'matches': [{'message': 'The usual collocation for “interest” is “in”, not “by”.',
   'shortMessage': 'Collocation error',
   'replacements': [{'value': 'interest in'}],
   'offset': 101,
   'length': 11,
   'context': {'text': '... the percentage of revenue generated by interest by 11.5996389009 in the year 1999.',
    'offset': 43,
    'length': 11},
   'sentence': 'The percentage of revenue generated by grants is greater than the percentage of revenue generated by interest by 11.5996389009 in the year 1999.',
   'type': {'typeName': 'Other'},
   'rule': {'i

In [21]:
import random

In [23]:
random.choices([c['caption'] for c in interest], k=5)

['The Revenue generated (in %) of Afghanistan in Grants is greater than that in Interest by a factor of 3655.7716.',
 'The Revenue generated (in %) of the Bahamas in Taxes is less than the Revenue generated (in %) of South Asia in Interest by 4.2596.',
 'The Revenue generated (in %) of Taxes is less than the Revenue generated (in %) of Interest by 5.5673 in 2010.',
 'In 2004, the Revenue generated (in %) of Grants is less than the Revenue generated (in %) of Interest by -11.7091.',
 'In 1996, the Revenue generated (in %) of Taxes is less than the Revenue generated (in %) of Interest by 8.0772.']

In [24]:
all(['interest by ' in c['caption'].lower() for c in interest])

True

In [29]:
in_interest = [re.sub(r'\b(?P<word>interest) by\b', r'\1 in', c['caption'], flags=re.IGNORECASE) for c in interest]

In [30]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(None, c, None) for c in in_interest)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 828 out of 828 | elapsed:    1.1s finished


In [31]:
res = [c for c in res if len(c['matches']) > 0]
res

[]

In [43]:
for doc in nlp.pipe([c['caption'] for c in interest]):
    root = [c for c in doc if c.dep_ == 'ROOT'][0]
    has_comparisons = len([c for c in root.children if c.dep_ == 'acomp']) > 0
    exceeds = root.text == 'exceeds'
    assert has_comparisons or exceeds

In [45]:
for t in interest:
    ig = t.get('ignore', ['INTERESTED_BY'])
    t['ignore'] = ig

In [46]:
set([len(t.get('ignore', [])) for t in interest])

{1}

In [47]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r['ignore']}}) for r in interest]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 828,
 'nModified': 828,
 'nRemoved': 0,
 'upserted': []}

In [48]:
ctr.most_common(5)

[('The singular proper name ‘Republic’ must be used with a third-person or a past tense verb.',
  1302),
 ('Possible typo: you repeated a word', 847),
 ('The usual collocation for “Interest” is “in”, not “by”.', 795),
 ('It seems like there are too many consecutive spaces here.', 610),
 ('With the plural noun ‘imports’, the verb inflection ‘is’ is not correct.',
  444)]

In [49]:
spaces = lt.filter_lt_messages(G, message='It seems like there are too many consecutive spaces here.')
spaces[0]

{'_id': 19528809,
 'question_string': 'In the year 1993, what is the difference between the Stocks(% of exports of goods, services and primary income) of Grenada and Stocks(% of exports of goods, services and primary income) of Jordan ?',
 'template': 'arithmetic',
 'answer': -129.6137,
 'type': 'dot_line',
 'template_id': 42,
 'caption': 'In 1993, the Stocks(% of exports of goods, services and primary income) of Grenada are less than the Stocks(% of exports of goods, services and primary income) of Jordan  by -129.6137.',
 'matches': [{'message': 'It seems like there are too many consecutive spaces here.',
   'shortMessage': '',
   'replacements': [{'value': ' '}],
   'offset': 169,
   'length': 2,
   'context': {'text': '..., services and primary income) of Jordan  by -129.6137.',
    'offset': 43,
    'length': 2},
   'sentence': 'In 1993, the Stocks(% of exports of goods, services and primary income) of Grenada are less than the Stocks(% of exports of goods, services and primary in

In [50]:
for s in spaces:
    s['caption'] = re.sub(r'\s+', ' ', s['caption'])

In [51]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(c['_id'], c['caption'], c.get('ignore')) for c in spaces)
res = [r for r in res if len(r['matches']) > 0]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
2023-03-21 12:40:44.680554: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-21 12:40:44.681279: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-21 12:40:44.681326: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-21 12:40:44.685469: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror

In [53]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'caption': r['caption']}}) for r in spaces]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 610,
 'nModified': 610,
 'nRemoved': 0,
 'upserted': []}

In [54]:
ctr.most_common(5)

[('The singular proper name ‘Republic’ must be used with a third-person or a past tense verb.',
  1302),
 ('Possible typo: you repeated a word', 847),
 ('The usual collocation for “Interest” is “in”, not “by”.', 795),
 ('It seems like there are too many consecutive spaces here.', 610),
 ('With the plural noun ‘imports’, the verb inflection ‘is’ is not correct.',
  444)]

In [55]:
is_imports = lt.filter_lt_messages(G, message='With the plural noun ‘imports’, the verb inflection ‘is’ is not correct.')
is_imports[0]

{'_id': 125456,
 'question_string': 'In the year 2007, what is the difference between the service imports in Israel and service imports in Croatia ?',
 'template': 'arithmetic',
 'answer': 13872692036.0,
 'type': 'dot',
 'template_id': 42,
 'caption': 'In 2007, the difference between the service imports in Israel and the service imports in Croatia is 13872692036.0.',
 'matches': [{'message': 'With the plural noun ‘imports’, the verb inflection ‘is’ is not correct.',
   'shortMessage': 'Agreement error',
   'replacements': [{'value': 'are'},
    {'value': 'was'},
    {'value': 'were',
     'shortDescription': "second-person singular simple past of 'be', e.g. 'you were'"}],
   'offset': 97,
   'length': 2,
   'context': {'text': '...rael and the service imports in Croatia is 13872692036.0.',
    'offset': 43,
    'length': 2},
   'sentence': 'In 2007, the difference between the service imports in Israel and the service imports in Croatia is 13872692036.0.',
   'type': {'typeName': 'Other

In [56]:
qids = [s['_id'] for s in is_imports]
sents = [s['caption'] for s in is_imports]

pairs = []
for doc, qid in zip(nlp.pipe(sents), qids):
    root = False
    for token in doc:
        if token.dep_ == 'ROOT':
            root = token
            break
    if root:
        try:
            child = [k for k in root.children if k.dep_ == 'nsubj'][0]
            pairs.append((qid, root.text, child.text))
        except IndexError:
            continue

pairs = pd.DataFrame(pairs, columns=['qid', 'root', 'nsubj'])
pairs.drop_duplicates(subset=['root', 'nsubj'])

,qid,root,nsubj
0,125456,is,difference
4,167074,is,sum
14,335816,is,mean


In [57]:
for t in is_imports:
    ig = t.get('ignore', [])
    if 'NNS_IN_NNP_VBZ' not in ig:
        ig.append('NNS_IN_NNP_VBZ')
    t['ignore'] = ig

    
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r['ignore']}}) for r in is_imports]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 444,
 'nModified': 444,
 'nRemoved': 0,
 'upserted': []}

In [58]:
ctr.most_common(6)

[('The singular proper name ‘Republic’ must be used with a third-person or a past tense verb.',
  1302),
 ('Possible typo: you repeated a word', 847),
 ('The usual collocation for “Interest” is “in”, not “by”.', 795),
 ('It seems like there are too many consecutive spaces here.', 610),
 ('With the plural noun ‘imports’, the verb inflection ‘is’ is not correct.',
  444),
 ('With the plural noun ‘gains’, the verb inflection ‘is’ is not correct.',
  416)]

In [59]:
gains = lt.filter_lt_messages(G, message='With the plural noun ‘gains’, the verb inflection ‘is’ is not correct.')
gains[0]

{'_id': 623101,
 'question_string': 'What is the percentage of revenue obtained from taxes on profits and capital gains in Cameroon ?',
 'template': 'data_retrieval',
 'answer': 57.2657170331,
 'type': 'hbar',
 'template_id': 2,
 'caption': 'The percentage of revenue obtained from taxes on profits and capital gains in Cameroon is 57.2657170331.',
 'matches': [{'message': 'With the plural noun ‘gains’, the verb inflection ‘is’ is not correct.',
   'shortMessage': 'Agreement error',
   'replacements': [{'value': 'are'},
    {'value': 'was'},
    {'value': 'were',
     'shortDescription': "second-person singular simple past of 'be', e.g. 'you were'"}],
   'offset': 87,
   'length': 2,
   'context': {'text': '...n profits and capital gains in Cameroon is 57.2657170331.',
    'offset': 43,
    'length': 2},
   'sentence': 'The percentage of revenue obtained from taxes on profits and capital gains in Cameroon is 57.2657170331.',
   'type': {'typeName': 'Other'},
   'rule': {'id': 'NNS_IN_NNP

In [60]:
qids = [s['_id'] for s in gains]
sents = [s['caption'] for s in gains]

pairs = []
for doc, qid in zip(nlp.pipe(sents), qids):
    root = False
    for token in doc:
        if token.dep_ == 'ROOT':
            root = token
            break
    if root:
        try:
            child = [k for k in root.children if k.dep_ == 'nsubj'][0]
            pairs.append((qid, root.text, child.text))
        except IndexError:
            continue

pairs = pd.DataFrame(pairs, columns=['qid', 'root', 'nsubj'])
pairs.drop_duplicates(subset=['root', 'nsubj'])

,qid,root,nsubj
0,623101,is,percentage
25,19533856,is,Tax
29,19533890,is,difference


In [61]:
for t in gains:
    ig = t.get('ignore', [])
    if 'NNS_IN_NNP_VBZ' not in ig:
        ig.append('NNS_IN_NNP_VBZ')
    t['ignore'] = ig

    
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r['ignore']}}) for r in gains]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 416,
 'nModified': 416,
 'nRemoved': 0,
 'upserted': []}

In [62]:
ctr.most_common(7)

[('The singular proper name ‘Republic’ must be used with a third-person or a past tense verb.',
  1302),
 ('Possible typo: you repeated a word', 847),
 ('The usual collocation for “Interest” is “in”, not “by”.', 795),
 ('It seems like there are too many consecutive spaces here.', 610),
 ('With the plural noun ‘imports’, the verb inflection ‘is’ is not correct.',
  444),
 ('With the plural noun ‘gains’, the verb inflection ‘is’ is not correct.',
  416),
 ('With the plural noun ‘services’, the verb inflection ‘exceeds’ is not correct.',
  414)]

In [63]:
services = lt.filter_lt_messages(G, message='With the plural noun ‘services’, the verb inflection ‘exceeds’ is not correct.')
services[0]

{'_id': 603866,
 'question_string': 'What is the difference between the percentage of self-employed male workers in Bahamas and the percentage of male workers in services in Norway ?',
 'template': 'arithmetic',
 'answer': -42.9999990463,
 'type': 'hbar',
 'template_id': 39,
 'caption': 'The percentage of male workers in services in Norway exceeds the percentage of self-employed male workers in the Bahamas by -42.9999990463.',
 'matches': [{'message': 'With the plural noun ‘services’, the verb inflection ‘exceeds’ is not correct.',
   'shortMessage': 'Agreement error',
   'replacements': [{'value': 'exceed', 'shortDescription': 'surpass'},
    {'value': 'exceeded'}],
   'offset': 53,
   'length': 7,
   'context': {'text': '...e of male workers in services in Norway exceeds the percentage of self-employed male wo...',
    'offset': 43,
    'length': 7},
   'sentence': 'The percentage of male workers in services in Norway exceeds the percentage of self-employed male workers in the Bahama

In [64]:
qids = [s['_id'] for s in services]
sents = [s['caption'] for s in services]

pairs = []
for doc, qid in zip(nlp.pipe(sents), qids):
    root = False
    for token in doc:
        if token.dep_ == 'ROOT':
            root = token
            break
    if root:
        try:
            child = [k for k in root.children if k.dep_ == 'nsubj'][0]
            pairs.append((qid, root.text, child.text))
        except IndexError:
            continue

pairs = pd.DataFrame(pairs, columns=['qid', 'root', 'nsubj'])
pairs.drop_duplicates(subset=['root', 'nsubj'])

,qid,root,nsubj
0,603866,exceeds,percentage
16,977344,exceeds,balance
29,11474765,exceeds,Balance
47,11624179,exceeds,%
69,12325411,exceeds,Revenue


In [65]:
for t in services:
    ig = t.get('ignore', [])
    if 'NNS_IN_NNP_VBZ' not in ig:
        ig.append('NNS_IN_NNP_VBZ')
    t['ignore'] = ig

    
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r['ignore']}}) for r in services]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 414,
 'nModified': 414,
 'nRemoved': 0,
 'upserted': []}

In [66]:
ctr.most_common(10)

[('The singular proper name ‘Republic’ must be used with a third-person or a past tense verb.',
  1302),
 ('Possible typo: you repeated a word', 847),
 ('The usual collocation for “Interest” is “in”, not “by”.', 795),
 ('It seems like there are too many consecutive spaces here.', 610),
 ('With the plural noun ‘imports’, the verb inflection ‘is’ is not correct.',
  444),
 ('With the plural noun ‘gains’, the verb inflection ‘is’ is not correct.',
  416),
 ('With the plural noun ‘services’, the verb inflection ‘exceeds’ is not correct.',
  414),
 ('Possible agreement error. The noun ‘amount’ seems to be countable.', 342),
 ('Did you just mean “,” or “.”?', 337),
 ('Possible spelling mistake. ‘Programmes’ is British English.', 330)]

In [67]:
amount = lt.filter_lt_messages(G, message='Possible agreement error. The noun ‘amount’ seems to be countable.')
amount[0]

{'_id': 2459,
 'question_string': 'What is the difference between the highest and the second-highest amount earned from national visitors ?',
 'template': 'compound',
 'answer': 155000000,
 'type': 'dot',
 'template_id': 15,
 'caption': 'The top two amount earned from national visitors differs by 155000000.',
 'matches': [{'message': 'Possible agreement error. The noun ‘amount’ seems to be countable.',
   'shortMessage': 'Grammatical problem',
   'replacements': [{'value': 'amounts'}],
   'offset': 12,
   'length': 6,
   'context': {'text': 'The top two amount earned from national visitors differs b...',
    'offset': 12,
    'length': 6},
   'sentence': 'The top two amount earned from national visitors differs by 155000000.',
   'type': {'typeName': 'Other'},
   'rule': {'id': 'CD_NN',
    'subId': '1',
    'sourceFile': 'grammar.xml',
    'description': 'Possible agreement error: numeral + singular countable noun',
    'issueType': 'grammar',
    'category': {'id': 'GRAMMAR', 'name':

In [72]:
for c in amount:
    c['caption'] = c['caption'].replace('The top two amount', 'The top two amounts')

In [76]:
for c in amount:
    c['caption'] = c['caption'].replace('differs by', 'differ by')

In [78]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(c['_id'], c['caption'], c.get('ignore')) for c in amount)
res = [r for r in res if len(r['matches']) > 0]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
2023-03-21 13:20:46.430879: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-21 13:20:46.431214: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-21 13:20:46.431225: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-21 13:20:46.449369: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror

In [79]:
res

[]

In [80]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'caption': r['caption']}}) for r in amount]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 342,
 'nModified': 342,
 'nRemoved': 0,
 'upserted': []}

In [81]:
comma = lt.filter_lt_messages(G, message='Did you just mean “,” or “.”?')
comma[0]

{'_id': 19498351,
 'question_string': 'What is the difference between the Number of troops, police, and military observers in mandate in 2012 and 2013 ?',
 'template': 'arithmetic',
 'answer': -20.0,
 'type': 'dot_line',
 'template_id': 44,
 'caption': 'The military observers in mandate are -20.0 greater than the Number of troops, police,.',
 'matches': [{'message': 'Did you just mean “,” or “.”?',
   'shortMessage': '',
   'replacements': [{'value': ','}, {'value': '.'}],
   'offset': 85,
   'length': 2,
   'context': {'text': '...reater than the Number of troops, police,.',
    'offset': 43,
    'length': 2},
   'sentence': 'The military observers in mandate are -20.0 greater than the Number of troops, police,.',
   'type': {'typeName': 'Other'},
   'rule': {'id': 'COMMA_PERIOD',
    'subId': '1',
    'sourceFile': 'grammar.xml',
    'description': 'redundant punctuation',
    'issueType': 'typographical',
    'category': {'id': 'PUNCTUATION', 'name': 'Punctuation'}},
   'ignoreForIn

In [83]:
for c in comma:
    assert c['caption'].endswith(',.')

In [84]:
for c in comma:
    c['caption'] = c['caption'].replace(',.', '.')

In [85]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(c['_id'], c['caption'], c.get('ignore')) for c in comma)
res = [r for r in res if len(r['matches']) > 0]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 255 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 337 out of 337 | elapsed:    0.4s finished


In [86]:
res

[]

In [87]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'caption': r['caption']}}) for r in comma]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 337,
 'nModified': 337,
 'nRemoved': 0,
 'upserted': []}

In [89]:
ctr.most_common(15)

[('The singular proper name ‘Republic’ must be used with a third-person or a past tense verb.',
  1302),
 ('Possible typo: you repeated a word', 847),
 ('The usual collocation for “Interest” is “in”, not “by”.', 795),
 ('It seems like there are too many consecutive spaces here.', 610),
 ('With the plural noun ‘imports’, the verb inflection ‘is’ is not correct.',
  444),
 ('With the plural noun ‘gains’, the verb inflection ‘is’ is not correct.',
  416),
 ('With the plural noun ‘services’, the verb inflection ‘exceeds’ is not correct.',
  414),
 ('Possible agreement error. The noun ‘amount’ seems to be countable.', 342),
 ('Did you just mean “,” or “.”?', 337),
 ('Possible spelling mistake. ‘Programmes’ is British English.', 330),
 ('This phrase is duplicated. You should probably use “aid received” only once.',
  304),
 ('With the plural noun ‘services’, the verb inflection ‘is’ is not correct.',
  294),
 ('With the plural noun ‘subscribers’, the verb inflection ‘is’ is not correct.',
  

In [91]:
aid = lt.filter_lt_messages(G, message='This phrase is duplicated. You should probably use “aid received” only once.')
aid[0]

{'_id': 19532942,
 'question_string': 'What is the average Aid received (current US$) of Kiribati per Type of aid received ?',
 'template': 'arithmetic',
 'answer': 4030000.0,
 'type': 'vbar',
 'template_id': 35,
 'caption': 'The average per Type of aid received Aid received (current USD) of Kiribati is 4030000.0.',
 'matches': [{'message': 'This phrase is duplicated. You should probably use “aid received” only once.',
   'shortMessage': 'Duplicated phrase',
   'replacements': [{'value': 'aid received'}],
   'offset': 24,
   'length': 25,
   'context': {'text': 'The average per Type of aid received Aid received (current USD) of Kiribati is 4030000.0....',
    'offset': 24,
    'length': 25},
   'sentence': 'The average per Type of aid received Aid received (current USD) of Kiribati is 4030000.0.',
   'type': {'typeName': 'Other'},
   'rule': {'id': 'PHRASE_REPETITION',
    'subId': '1',
    'sourceFile': 'grammar.xml',
    'description': "Repetition of two words ('at the at the')",
   

In [92]:
random.choice(aid)['caption']

'The mean per Type of aid received Aid received (constant 2012 the US$) of Swaziland is 41370000.0.'

In [93]:
for c in aid:
    c['caption'] = c['caption'].replace('aid received Aid received', 'aid received')

In [94]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(c['_id'], c['caption'], c.get('ignore')) for c in aid)
res = [r for r in res if len(r['matches']) > 0]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 258 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 304 out of 304 | elapsed:    0.4s finished


In [95]:
res

[]

In [96]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'caption': r['caption']}}) for r in aid]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 304,
 'nModified': 304,
 'nRemoved': 0,
 'upserted': []}

In [101]:
verb_infl = [
    'With the plural noun ‘services’, the verb inflection ‘is’ is not correct.',
    'With the plural noun ‘subscribers’, the verb inflection ‘is’ is not correct.',
    'With the plural noun ‘metals’, the verb inflection ‘is’ is not correct.'
    'With the plural noun ‘institutions’, the verb inflection ‘is’ is not correct.'
]
X = []
for msg in verb_infl:
    X.extend(lt.filter_lt_messages(G, message=msg))
verb_infl = X

In [102]:
qids = [s['_id'] for s in verb_infl]
sents = [s['caption'] for s in verb_infl]

pairs = []
for doc, qid in zip(nlp.pipe(sents), qids):
    root = False
    for token in doc:
        if token.dep_ == 'ROOT':
            root = token
            break
    if root:
        try:
            child = [k for k in root.children if k.dep_ == 'nsubj'][0]
            pairs.append((qid, root.text, child.text))
        except IndexError:
            continue

pairs = pd.DataFrame(pairs, columns=['qid', 'root', 'nsubj'])
pairs.drop_duplicates(subset=['root', 'nsubj'])

,qid,root,nsubj
0,11624155,is,%
162,1862948,is,computer
167,3115452,is,OECD
293,19500552,is,difference


In [104]:
pairs = pairs[pairs['nsubj'].isin(['%', 'difference'])]

In [105]:
pairs.drop_duplicates(subset=['root', 'nsubj'])

,qid,root,nsubj
0,11624155,is,%
293,19500552,is,difference


In [107]:
to_ignore = set(pairs['qid'].values)

In [108]:
to_ignore = [c for c in verb_infl if c['_id'] in to_ignore]

In [110]:
for t in to_ignore:
    ig = t.get('ignore', [])
    if 'NNS_IN_NNP_VBZ' not in ig:
        ig.append('NNS_IN_NNP_VBZ')
    t['ignore'] = ig

    
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r['ignore']}}) for r in to_ignore]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 567,
 'nModified': 567,
 'nRemoved': 0,
 'upserted': []}

In [123]:
ctr.most_common(50)

[('The singular proper name ‘Republic’ must be used with a third-person or a past tense verb.',
  1302),
 ('Possible typo: you repeated a word', 847),
 ('The usual collocation for “Interest” is “in”, not “by”.', 795),
 ('It seems like there are too many consecutive spaces here.', 610),
 ('With the plural noun ‘imports’, the verb inflection ‘is’ is not correct.',
  444),
 ('With the plural noun ‘gains’, the verb inflection ‘is’ is not correct.',
  416),
 ('With the plural noun ‘services’, the verb inflection ‘exceeds’ is not correct.',
  414),
 ('Possible agreement error. The noun ‘amount’ seems to be countable.', 342),
 ('Did you just mean “,” or “.”?', 337),
 ('Possible spelling mistake. ‘Programmes’ is British English.', 330),
 ('This phrase is duplicated. You should probably use “aid received” only once.',
  304),
 ('With the plural noun ‘services’, the verb inflection ‘is’ is not correct.',
  294),
 ('With the plural noun ‘subscribers’, the verb inflection ‘is’ is not correct.',
  

In [120]:
repeat = lt.filter_lt_messages(G, message='You repeated a verb. Please check.')
repeat[0]

{'_id': 19569714,
 'question_string': 'What is the average Tax (as % of total tax revenue) in Cameroon per Taxes ?',
 'template': 'arithmetic',
 'answer': 14.9575,
 'type': 'hbar',
 'template_id': 35,
 'caption': 'The average per Taxes Tax (as % of total tax revenue) in Cameroon is 14.9575.',
 'matches': [{'message': 'You repeated a verb. Please check.',
   'shortMessage': '',
   'replacements': [{'value': 'Taxes'}, {'value': 'Tax'}],
   'offset': 16,
   'length': 9,
   'context': {'text': 'The average per Taxes Tax (as % of total tax revenue) in Cameroon...',
    'offset': 16,
    'length': 9},
   'sentence': 'The average per Taxes Tax (as % of total tax revenue) in Cameroon is 14.9575.',
   'type': {'typeName': 'Other'},
   'rule': {'id': 'REPEATED_VERBS',
    'subId': '1',
    'sourceFile': 'grammar.xml',
    'description': 'Repeated verb forms',
    'issueType': 'grammar',
    'category': {'id': 'GRAMMAR', 'name': 'Grammar'}},
   'ignoreForIncompleteSentence': True,
   'contextForS

In [121]:
lt.get_repl_values(repeat)

{'Tax', 'Taxes'}

In [122]:
for t in repeat:
    ig = t.get('ignore', [])
    if 'REPEATED_VERBS' not in ig:
        ig.append('REPEATED_VERBS')
    t['ignore'] = ig

    
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r['ignore']}}) for r in repeat]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 119,
 'nModified': 119,
 'nRemoved': 0,
 'upserted': []}

In [115]:
for c in rate:
    c['caption'] = c['caption'].replace('rate differ', 'rates differ')

In [116]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'caption': r['caption']}}) for r in rate]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 242,
 'nModified': 242,
 'nRemoved': 0,
 'upserted': []}

In [117]:
imp = lt.filter_lt_messages(G, message='You should probably use “import”.')
imp[0]

{'_id': 13408,
 'question_string': 'Across all years, what is the maximum arms imports ?',
 'template': 'min_max',
 'answer': '235000000',
 'type': 'dot',
 'template_id': 22,
 'caption': 'The greatest arms imports across all years are 235000000.',
 'matches': [{'message': 'You should probably use “import”.',
   'shortMessage': 'Possible agreement error',
   'replacements': [{'value': 'import'}],
   'offset': 18,
   'length': 7,
   'context': {'text': 'The greatest arms imports across all years are 235000000.',
    'offset': 18,
    'length': 7},
   'sentence': 'The greatest arms imports across all years are 235000000.',
   'type': {'typeName': 'Other'},
   'rule': {'id': 'AGREEMENT_SENT_START',
    'subId': '4',
    'sourceFile': 'grammar.xml',
    'description': 'Possible agreement error: plural noun + singular verb',
    'issueType': 'grammar',
    'category': {'id': 'GRAMMAR', 'name': 'Grammar'}},
   'ignoreForIncompleteSentence': True,
   'contextForSureMatch': 4}]}